# Modern Data Architectures Group Project

In [2]:
pip install yt-dlp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 35.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import yt_dlp
import os

FFMPEG_PATH = "/home/osbdet/ffmpeg"  # Adjust if needed

# Create download directory
download_path = "songs"
os.makedirs(download_path, exist_ok=True)

def download_song(song_name):
    """Download a song from YouTube and force MP3 conversion"""
    ydl_opts = {
        'format': 'bestaudio/best',
        'default_search': 'ytsearch1',
        'noplaylist': True,
        'outtmpl': f'{download_path}/%(title)s.%(ext)s',
        'ffmpeg_location': FFMPEG_PATH,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'quiet': False  # You can set this to True if you want cleaner output
    }
    
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([song_name])
    
    print(f"[INFO] Downloaded and converted: {song_name}")

try:
    with open("songs_list.txt", "r") as f:
        songs = [line.strip() for line in f if line.strip()]
except FileNotFoundError:
    print("[ERROR] songs_list.txt not found in the current directory.")
    songs = []

for song in songs:
    try:
        download_song(song)
    except Exception as e:
        print(f"[WARNING] Skipped due to error: {song}")
        print(e)

ModuleNotFoundError: No module named 'yt_dlp'

In [2]:
pwd

'/home/osbdet/notebooks/spark/Modern-Data-Architectures'

In [2]:
import os
import subprocess

# FFMPEG_PATH = "/home/osbdet/ffmpeg"  # Ensure this path is correct

# Define input (where your WEBM files are) and output folders
input_folder = "/home/osbdet/notebooks/spark/Modern-Data-Architectures/songs"  
output_folder = "/home/osbdet/notebooks/spark/Modern-Data-Architectures/songs_mp3"
os.makedirs(output_folder, exist_ok=True)  

def convert_webm_to_mp3():
    """Convert all .webm files in a folder to .mp3"""
    for file in os.listdir(input_folder):
        if file.endswith(".webm"):
            input_path = os.path.join(input_folder, file)
            output_path = os.path.join(output_folder, file.replace(".webm", ".mp3"))
            
            # Use full path to FFmpeg
            command = ["ffmpeg", "-i", input_path, "-vn", "-ab", "192k", output_path]
            
            # Run FFmpeg command
            try:
                subprocess.run(command, check=True)
                print(f"Converted: {file} -> {output_path}")
            except FileNotFoundError:
                print(f"Error: FFmpeg not found at {FFMPEG_PATH}")
                break
            except subprocess.CalledProcessError as e:
                print(f"FFmpeg Error: {e}")
                break

convert_webm_to_mp3()

ffmpeg version 5.1.6-0+deb12u1 Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 12 (Debian 12.2.0-14)
  configuration: --prefix=/usr --extra-version=0+deb12u1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librist --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtheora --enable-libtwolame --enable-libvidstab --enab

✅ Converted: Michael Jackson - Billie Jean (Official Video).webm -> /home/osbdet/notebooks/spark/Modern-Data-Architectures/songs_mp3/Michael Jackson - Billie Jean (Official Video).mp3


size=    6935kB time=00:04:55.85 bitrate= 192.0kbits/s speed=55.1x    
video:0kB audio:6934kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.008745%


In [4]:
pip install librosa

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 58.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 39.1 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import librosa
import librosa.display
import numpy as np
import os
import matplotlib.pyplot as plt

# Define folder containing MP3 files
input_folder = "songs_mp3"  
output_folder = "fingerprints"
os.makedirs(output_folder, exist_ok=True)  

def generate_fingerprint(file_path):
    """Extracts a fingerprint (spectrogram peaks) from an audio file"""
    try:
        # Load audio file
        y, sr = librosa.load(file_path, sr=44100)
        
        # Compute Spectrogram
        D = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)
        
        # Get peaks in the spectrogram (Shazam-like fingerprinting)
        peaks = np.argwhere(D > np.percentile(D, 95))  # Extract top 5% peaks
        
        # Convert peaks into a unique fingerprint
        fingerprint = peaks.tolist()  # Convert to list format
        
        # Save fingerprint as a NumPy file
        fingerprint_file = os.path.join(output_folder, os.path.basename(file_path).replace(".mp3", ".npy"))
        np.save(fingerprint_file, fingerprint)
        
        print(f"Fingerprint created for {os.path.basename(file_path)}")

        plt.figure(figsize=(10, 6))
        librosa.display.specshow(D, sr=sr, x_axis="time", y_axis="log")
        plt.colorbar(format="%+2.0f dB")
        plt.title(f"Spectrogram - {os.path.basename(file_path)}")
        plt.savefig(f"{file_path.replace('.mp3', '.png')}")
        plt.close()

    except Exception as e:
        print(f"Error processing {file_path}: {e}")

# Process all MP3 files in the folder
for file in os.listdir(input_folder):
    if file.endswith(".mp3"):
        file_path = os.path.join(input_folder, file)
        generate_fingerprint(file_path)

✅ Fingerprint created for Michael Jackson - Billie Jean (Official Video).mp3


In [6]:
import librosa
import numpy as np
import os

# Define folders
songs_folder = "songs_mp3"  # Full song database
fingerprints_folder = "fingerprints"  # Precomputed fingerprints

def generate_snippet_fingerprint(file_path, start_time=10, duration=5):
    """Extracts a fingerprint from a short audio snippet"""
    try:
        # Load only a small snippet of the audio file
        y, sr = librosa.load(file_path, sr=44100, offset=start_time, duration=duration)
        
        # Compute spectrogram
        D = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)
        
        # Get peaks (top 5% of intensities)
        peaks = np.argwhere(D > np.percentile(D, 95))
        
        return peaks  # Return the fingerprint

    except Exception as e:
        print(f"Error processing snippet: {e}")
        return None

def find_closest_match(snippet_fingerprint):
    """Find the song with the closest matching fingerprint"""
    best_match = None
    best_score = float('inf')  # Lower score means better match

    for fingerprint_file in os.listdir(fingerprints_folder):
        if fingerprint_file.endswith(".npy"):
            song_fingerprint = np.load(os.path.join(fingerprints_folder, fingerprint_file))
            
            # Compare snippet fingerprint to full song fingerprint
            difference = len(set(map(tuple, snippet_fingerprint)) - set(map(tuple, song_fingerprint)))

            if difference < best_score:  # Lower difference means better match
                best_score = difference
                best_match = fingerprint_file.replace(".npy", "")

    return best_match if best_match else "No match found"

# Test with a snippet from a song
test_song = os.path.join(songs_folder, "Michael Jackson - Billie Jean (Official Video).mp3")  # Change this to an actual song in your database
snippet_fingerprint = generate_snippet_fingerprint(test_song)

if snippet_fingerprint is not None:
    match = find_closest_match(snippet_fingerprint)
    print(f"Closest match: {match}")

🔍 Closest match: Michael Jackson - Billie Jean (Official Video)
